In [1]:
import pandas as pd
from pyhive import hive
import re

In [8]:
cursor = hive.connect('localhost').cursor()
sql = """
SELECT SK.stk_cd, sk.stk_nm, sk.sec_nm, sk.ex_cd, 
	   hd.dt, hd.o_prc, hd.l_prc, hd.h_prc, hd.c_prc, hd.vol, 
	   M5_PRC, M10_PRC, M20_PRC, fy.fin_itm_val 
FROM stock sk
INNER JOIN history_dt hd 
ON (HD.stk_cd = SK.stk_cd)
INNER JOIN finance_y fy 
ON (hd.stk_cd = FY.stk_cd) 
AND SK.EX_CD = 'KP' 
AND HD.DT = '2019-01-04' 
ANd FY.FIN_ITM_NM = '주당순이익' 
AND FY.YY = '2018'
"""
cursor.execute(sql)
result = cursor.fetchall()

In [22]:
# 칼럼 정보 추출

# 조회할 칼럼정보 조회
desc = cursor.description
# 리스트의 행열 방향을 반대로 바꾸고  칼럼의 이름 추출
column_name = list(zip(*desc))[0]

# for element in column_name: column_name에서 칼럼 이름 1개를 element에 대입
# element.upper
# re.sub 특정 분자를 착아서 삭제
# \S: 문자열
# \. : .
# . 앞의 문자를 전부 삭제
# re.sub('\S*\.',"","FY.FIN_ITM_VAL")

column_name = [re.sub('\S*\.',"",element.upper()) for element in column_name]
column_name

['STK_CD',
 'STK_NM',
 'SEC_NM',
 'EX_CD',
 'DT',
 'O_PRC',
 'L_PRC',
 'H_PRC',
 'C_PRC',
 'VOL',
 'M5_PRC',
 'M10_PRC',
 'M20_PRC',
 'FIN_ITM_VAL']

In [35]:
# 데이터 프레임화
df = pd.DataFrame(result)
df.columns = column_name
# 데이터 정리
df["DT"] = pd.to_datetime(df["DT"])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 511 entries, 0 to 510
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   STK_CD       511 non-null    object        
 1   STK_NM       511 non-null    object        
 2   SEC_NM       511 non-null    object        
 3   EX_CD        511 non-null    object        
 4   DT           511 non-null    datetime64[ns]
 5   O_PRC        511 non-null    object        
 6   L_PRC        511 non-null    object        
 7   H_PRC        511 non-null    object        
 8   C_PRC        511 non-null    object        
 9   VOL          511 non-null    object        
 10  M5_PRC       511 non-null    object        
 11  M10_PRC      511 non-null    object        
 12  M20_PRC      511 non-null    object        
 13  FIN_ITM_VAL  511 non-null    object        
dtypes: datetime64[ns](1), object(13)
memory usage: 56.0+ KB


In [36]:
# PER 주가 순익비율
# PER = 주식가격 / 주당 순이익(EPS)

In [39]:
df.rename(columns={"FIN_ITM_VAL":"주당순이익"}, inplace=True)
df["PER"] = df["C_PRC"] / df["주당순이익"]
df[["STK_CD", "STK_NM","PER"]]

,STK_CD,STK_NM,PER
0,000020,동화약품,26.10958904109589041095890411
1,000040,KR모터스,-10.80423280423280423280423280
2,000050,경방,14.86486486486486486486486486
3,000070,삼양홀딩스,8.813446864978436359615171956
4,000080,하이트진로,52.82131661442006269592476489
...,...,...,...
506,286940,롯데정보통신,16.78004535147392290249433107
507,298000,효성화학,22.30193387398627573300062383
508,298020,효성티앤씨,37.50762350071152673307582842
509,298040,효성중공업,191.4215686274509803921568627


In [40]:
# 주식 봉 만들기

In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyhive import hive
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [44]:
plt.style.use("ggplot")

In [66]:
cursor = hive.connect("localhost").cursor()
sql = """
SELECT hd.stk_cd, hd.dt, hd.o_prc, hd.c_prc, hd.l_prc, hd.h_prc, hd.vol 
FROM history_dt hd
WHERE hd.stk_cd = '005930'
AND hd.dt >= '2019-01-01'
AND hd.dt <= '2019-12-31'
"""
cursor.execute(sql)
result = cursor.fetchall()
result

[('005930',
  '2019-01-02',
  Decimal('39400.000'),
  Decimal('38750.000'),
  Decimal('38550.000'),
  Decimal('39400.000'),
  Decimal('7847664.000')),
 ('005930',
  '2019-01-03',
  Decimal('38300.000'),
  Decimal('37600.000'),
  Decimal('37450.000'),
  Decimal('38550.000'),
  Decimal('12471493.000')),
 ('005930',
  '2019-01-04',
  Decimal('37450.000'),
  Decimal('37450.000'),
  Decimal('36850.000'),
  Decimal('37600.000'),
  Decimal('14108958.000')),
 ('005930',
  '2019-01-07',
  Decimal('38000.000'),
  Decimal('38750.000'),
  Decimal('37800.000'),
  Decimal('38900.000'),
  Decimal('12748997.000')),
 ('005930',
  '2019-01-08',
  Decimal('38000.000'),
  Decimal('38100.000'),
  Decimal('37950.000'),
  Decimal('39200.000'),
  Decimal('12756554.000')),
 ('005930',
  '2019-01-09',
  Decimal('38650.000'),
  Decimal('39600.000'),
  Decimal('38300.000'),
  Decimal('39600.000'),
  Decimal('17452708.000')),
 ('005930',
  '2019-01-10',
  Decimal('40000.000'),
  Decimal('39800.000'),
  Decimal('39

In [67]:
# 조회할 칼럼정보 조회
desc = cursor.description
# 리스트의 행열 방향을 반대로 바꾸고  칼럼의 이름 추출
column_name = list(zip(*desc))[0]

# for element in column_name: column_name에서 칼럼 이름 1개를 element에 대입
# element.upper
# re.sub 특정 분자를 착아서 삭제
# \S: 문자열
# \. : .
# . 앞의 문자를 전부 삭제
# re.sub('\S*\.',"","FY.FIN_ITM_VAL")

column_name = [re.sub('\S*\.',"",element.upper()) for element in column_name]
column_name

['STK_CD', 'DT', 'O_PRC', 'C_PRC', 'L_PRC', 'H_PRC', 'VOL']

In [71]:
# 데이터 프레임화
df = pd.DataFrame(result)

# 목차적용
df.columns = column_name

# 데이터 정리
df["DT"] = pd.to_datetime(df["DT"])
# C_PRC 칼럼부터 마지막 칼럼까지 타입을 정수로 변환
type_columns = ["C_PRC", "L_PRC", "H_PRC", "VOL"]
df[type_columns] = df[type_columns].astype("int64")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 246 entries, 0 to 245
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   STK_CD  246 non-null    object        
 1   DT      246 non-null    datetime64[ns]
 2   O_PRC   246 non-null    object        
 3   C_PRC   246 non-null    int64         
 4   L_PRC   246 non-null    int64         
 5   H_PRC   246 non-null    int64         
 6   VOL     246 non-null    int64         
dtypes: datetime64[ns](1), int64(4), object(2)
memory usage: 13.6+ KB


In [73]:
# 봉차트 그리기
candlestick = go.Candlestick(
    x = df["DT"],                       # X축 날짜
    open = df["O_PRC"],                 # 시가
    high = df["H_PRC"],                 # 고가
    low = df["L_PRC"],                  # 저가
    close = df["C_PRC"],                # 종가
    increasing_line_color = "red",      # 양봉색
    decreasing_line_color = "blue",     # 음봉색 설정
)

# 봉차트를 fig에 저장
fig  = go.Figure(data=[candlestick])

fig.update_layout(
    title = "2019년 삼성전자 주식가격",
    yaxis_title = "주식가격"
)

fig.show()

In [75]:
# 봉차트 그리기
candlestick = go.Candlestick(
    x = df["DT"],                       # X축 날짜
    open = df["O_PRC"],                 # 시가
    high = df["H_PRC"],                 # 고가
    low = df["L_PRC"],                  # 저가
    close = df["C_PRC"],                # 종가
    increasing_line_color = "red",      # 양봉색
    decreasing_line_color = "blue",     # 음봉색 설정
    showlegend=False                    # 별레 출력안함
)

# 거래량 막대 그래프
volume_bars = go.Bar(
    x = df["DT"],       # X좌표 날짜
    y = df["VOL"],      # y 좌표 거래량
    showlegend=False,   # 범례 출력 안함
    marker={
        "color": "rgba(128,128,128,0.5)" # 거래량 막대 차트 색 > 회색
    }
)

# 2줄을 2칼으로 나누어 그래프를 그림
fig = make_subplots(
    rows=2,
    cols=1,
    shared_xaxes=True,
    subplot_titles=("2019년 삼성전자 거래가격", "거래량")
)

fig.add_trace(candlestick, row=1, col=1)
fig.add_trace(volume_bars, row=2, col=1)
fig.update_layout(
    height=800, 
    width=1200, 
    title_text="삼성전자 봉차트"
)

fig.show()